In [1]:
%load_ext jupyter_black

In [2]:
from __future__ import annotations

import time
import itertools
import random
import numpy as np

from mesoscaler.generic import DataWorker, DataGenerator
from mesoscaler._typing import Array

In [3]:
Index = tuple[int, int, int, int]


DUMMY_DATA: dict[Index, np.ndarray] = {
    # some dummy data either on disk or in
    k: np.random.rand(100, 100)
    for k in itertools.product([1, 2, 3], [4, 5, 6], [4, 5, 6], [4, 5, 6])
}


def get_data_from_disk(key: Index) -> np.ndarray:
    time.sleep(0.1)  # io latency
    return DUMMY_DATA[key]

In [4]:
indices = random.choices(list(DUMMY_DATA.keys()), k=20)

print(len(DUMMY_DATA), len(indices))

81 20


The worker class is `Mapping` that is instantiated with a Iterable sequence of indices.

In [5]:
from numpy.typing import NDArray
from typing import NewType

_100 = NewType("100", int)  # type: ignore


class MyWorker(DataWorker[Index, Array[[_100, _100], np.float_]]):
    def __getitem__(self, idx: Index) -> Array[[_100, _100], np.float_]:
        return get_data_from_disk(idx)


worker = MyWorker(indices=indices)
data = worker[indices[0]]
print(worker, data.shape, sep="\n")

MyWorker(size=20):
- (3, 4, 4, 5): ndarray[(100, 100), dtype[float64]]
- (2, 6, 5, 5): ndarray[(100, 100), dtype[float64]]
- (2, 4, 4, 4): ndarray[(100, 100), dtype[float64]]
- (1, 4, 4, 6): ndarray[(100, 100), dtype[float64]]
- (2, 5, 4, 4): ndarray[(100, 100), dtype[float64]]
...
- (3, 4, 5, 4): ndarray[(100, 100), dtype[float64]]
(100, 100)


In [6]:
train, test = worker.split(0.8)
test

MyWorker[test](size=4):
- (3, 4, 6, 5): ndarray[(100, 100), dtype[float64]]
- (2, 5, 5, 5): ndarray[(100, 100), dtype[float64]]
- (3, 5, 6, 6): ndarray[(100, 100), dtype[float64]]
- (3, 4, 5, 4): ndarray[(100, 100), dtype[float64]]
...
- (3, 4, 5, 4): ndarray[(100, 100), dtype[float64]]

# DataConsumer

Assuming there is some IO bottle neck involved with loading data from disk the
`DataConsumer` can be used as a DataLoader that will queue up the data to be
loaded in the background while the model is training.

In [7]:
start = time.time()
for idx in worker:
    data = worker[idx]
    time.sleep(0.1)
print("worker:", time.time() - start)

start = time.time()
for x in DataGenerator(worker):
    time.sleep(0.1)
print("consumer:", time.time() - start)

worker: 4.006401062011719
consumer: 2.1048800945281982


Combining the Resampling pipeline with the data consumer.

In [8]:
import os
from mesoscaler.core import Mesoscale, P0, DependentDataset, ArrayProducer
import time
from mesoscaler.enums import (
    # - ERA5
    GEOPOTENTIAL,
    SPECIFIC_HUMIDITY,
    TEMPERATURE,
    U_COMPONENT_OF_WIND,
    V_COMPONENT_OF_WIND,
    # - URMA
    SURFACE_PRESSURE,
    TEMPERATURE_2M,
    SPECIFIC_HUMIDITY_2M,
    U_WIND_COMPONENT_10M,
    V_WIND_COMPONENT_10M,
    SURFACE_PRESSURE,
)

_test_data = "../tests/data"

urma_store = os.path.join(_test_data, "urma.zarr")
urma_dvars = [
    SURFACE_PRESSURE,
    TEMPERATURE_2M,
    SPECIFIC_HUMIDITY_2M,
    U_WIND_COMPONENT_10M,
    V_WIND_COMPONENT_10M,
]

era5_store = os.path.join(_test_data, "era5.zarr")
era5_dvars = [
    GEOPOTENTIAL,
    TEMPERATURE,
    SPECIFIC_HUMIDITY,
    U_COMPONENT_OF_WIND,
    V_COMPONENT_OF_WIND,
]

In [9]:
dx, dy = np.array([200, 175])
pressure_levels = [P0, 1000.0, 925.0, 850.0, 700.0, 500.0, 300.0]
scale = Mesoscale(dx, dy, pressure=pressure_levels, rate=15)

era5 = DependentDataset.from_zarr(era5_store, era5_dvars)  # get datasets
urma = DependentDataset.from_zarr(urma_store, urma_dvars)  # get datasets

In [10]:
N_SAMPLES = 5


start, stop = urma.time.to_numpy().astype("datetime64[h]")  # sample data only has 2 times


lons = np.random.choice(((urma.lons - 180) % 360 - 180).to_numpy().ravel(), N_SAMPLES)
lats = np.random.choice(urma.lats.to_numpy().ravel(), N_SAMPLES)

indices = zip(itertools.repeat(np.s_[start:stop]), np.c_[lons, lats].round(2))

worker = ArrayProducer.from_scale(indices, urma, era5, scale=scale)
worker

ArrayProducer(size=5):
- (2019-01-02T00:00:00Z:2019-01-02T01:00:00Z:None, [-84.37  49.05]): ndarray[(:, :, :, :, :), dtype[float64]]
- (2019-01-02T00:00:00Z:2019-01-02T01:00:00Z:None, [-93.65  52.58]): ndarray[(:, :, :, :, :), dtype[float64]]
- (2019-01-02T00:00:00Z:2019-01-02T01:00:00Z:None, [-67.99  49.10]): ndarray[(:, :, :, :, :), dtype[float64]]
- (2019-01-02T00:00:00Z:2019-01-02T01:00:00Z:None, [-78.04  23.00]): ndarray[(:, :, :, :, :), dtype[float64]]
- (2019-01-02T00:00:00Z:2019-01-02T01:00:00Z:None, [-61.71  43.92]): ndarray[(:, :, :, :, :), dtype[float64]]
...
- (2019-01-02T00:00:00Z:2019-01-02T01:00:00Z:None, [-61.71  43.92]): ndarray[(:, :, :, :, :), dtype[float64]]

In [11]:
idx = worker.indices[0]
worker[idx]

array([[[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          ...,
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

         [[ 1.23552771e+03,  1.23992542e+03,  1.23992542e+03, ...,
            1.13937268e+03,  1.13937268e+03,  1.13176587e+03],
          [ 1.23754822e+03,  1.23624084e+03,  1.23624084e+03, ...,
      

In [23]:
import functools
from mesoscaler._typing import Self
from typing import TypeVar

_T = TypeVar("_T")


class A:
    def __init__(self, x: int, *, world) -> None:
        pass

    @classmethod
    def partial(cls: type[_T], *args, **kwargs) -> functools.partial[_T]:
        return functools.partial(cls, *args, **kwargs)


def f(*, a:int, b:int, c:int) -> int:
    return a + b + c


c = functools.partial(f, a=1, b=2)
x = c(a=0, c=3)
x

5

In [30]:

class C:
    @functools.partialmethod
    def f(self):...

functools.partialmethod(<function <lambda> at 0x7f5e55803130>, , )

In [12]:
worker.get_array(idx)

<xarray.DataArray (variables: 5, T: 2, Z: 7, Y: 80, X: 80)>
array([[[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          ...,
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

         [[ 1.23552771e+03,  1.23992542e+03,  1.23992542e+03, ...,
            1.13937268e+03,  1.13937268e+03,  1.13176587e+03],
          [ 1.23754822e+03,  1.23624084e+03,  1.23624084e+03, ...,
            1.14115564e+03,  1.14115564e+03,  1.13984814e+03],
          [ 1.23754822e+03,  1.23624084e+03,  1.23624084e+03, ...,
            1.14115564e+03,  1.14115564e+03,  1.13984814e+03],
...
           -4.86295033e+00, -3.65440607e+00, -3.17796040e+00],
          [ 7.19427061e+00,  6.80580997e+00,  5.34493208e+00, ...,
           -5.30785370e+00, -5.09702253e+00, -3.76065159e+00],
          [ 7.62091303e+00,  8.14384079e+00,  7.54288912e+00, ...,
           -2.90902615e+00, -3.91669989e+00, -2.98206997e+00]],

         [[ 1.51408339e+01,  1.34787884e+01,  1.19695225e+01, ...,
            1.60528755e+01,  1.69463978e+01,  1.69024162e+01],
          [ 1.48283319e+01,  1.31408243e+01,  1.14810944e+01, ...,
            1.52519455e+01,  1.55760202e+01,  1.56801882e+01],
          [ 1.46917572e+01,  1.33954554e+01,  1.17542439e+01, ...,
            1.41408291e+01,  1.42704592e+01,  1.46292572e+01],
          ...,
          [ 1.02588663e+01,  1.08213692e+01,  1.12125750e+01, ...,
            2.50188637e+00,  4.35374689e+00,  4.21485710e+00],
          [ 9.66627121e+00,  1.01408100e+01,  1.07542391e+01, ...,
            1.52338475e-01,  3.46253920e+00,  1.15697277e+00],
          [ 9.08987999e+00,  9.36534405e+00,  9.27506542e+00, ...,
            8.88453007e-01,  2.75883198e+00, -1.76433718e+00]]]]],
      dtype=float32)
Coordinates:
  * variables  (variables) <U20 'specific_humidity_2m' ... 'v_wind_component_...
    vertical   (Z) float64 1.013e+03 1e+03 925.0 850.0 700.0 500.0 300.0
    time       (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
Dimensions without coordinates: T, Z, Y, X

In [13]:
worker.get_dataset(idx)

<xarray.Dataset>
Dimensions:               (T: 2, Z: 7, Y: 80, X: 80)
Coordinates:
    vertical              (Z) float64 1.013e+03 1e+03 925.0 ... 500.0 300.0
    time                  (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
Dimensions without coordinates: T, Z, Y, X
Data variables:
    specific_humidity_2m  (T, Z, Y, X) float32 0.0 0.0 0.0 ... 242.5 243.1 243.1
    surface_pressure      (T, Z, Y, X) float32 0.0 0.0 0.0 ... 6.028 8.985 6.694
    temperature_2m        (T, Z, Y, X) float32 0.0 0.0 0.0 ... 5.314 7.49 7.664
    u_wind_component_10m  (T, Z, Y, X) float32 0.0 0.0 0.0 ... 0.8514 -2.306
    v_wind_component_10m  (T, Z, Y, X) float32 0.0 0.0 0.0 ... 2.759 -1.764

In [14]:
TIME2TRAIN = 1.5

start = time.time()
for idx in worker:
    sample = worker[idx]
    print(sample.shape)
    time.sleep(TIME2TRAIN)
total = time.time() - start
print(
    f"""\
total = {total}
io_time = {total - TIME2TRAIN * N_SAMPLES}
"""
)

start = time.time()
for sample in DataGenerator(worker):
    print(sample.shape)
    time.sleep(TIME2TRAIN)
total = time.time() - start
print(
    f"""\
total = {total}
io_time = {total - TIME2TRAIN * N_SAMPLES}
"""
)

(5, 2, 7, 80, 80)
(5, 2, 7, 80, 80)
(5, 2, 7, 80, 80)
(5, 2, 7, 80, 80)
(5, 2, 7, 80, 80)
total = 15.167352676391602
io_time = 7.667352676391602

(5, 2, 7, 80, 80)
(5, 2, 7, 80, 80)
(5, 2, 7, 80, 80)
(5, 2, 7, 80, 80)
(5, 2, 7, 80, 80)
total = 9.311925172805786
io_time = 1.8119251728057861



In [15]:
from torch.utils.data import DataLoader


for batch in DataLoader(DataGenerator(worker), batch_size=2):
    print(batch.shape)

torch.Size([2, 5, 2, 7, 80, 80])
torch.Size([2, 5, 2, 7, 80, 80])
torch.Size([1, 5, 2, 7, 80, 80])
